In [87]:
import numpy as np
import pandas as pd
import geopandas as gpd

from geopandas.tools import geocode

import rtree
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

In [75]:
result = geocode("The Great Pyramid of Giza", provider="nominatim")
result

,geometry,address
0,POINT (31.13424 29.97913),"هرم خوفو, Cause way, كوم الأخضر, الجيزة, محافظ..."


In [76]:
point = result.geometry.iloc[0]
print("Latitude:", point.y)
print("Longitude:", point.x)

Latitude: 29.9791264
Longitude: 31.1342383751015


Snagged the top 100 universities in Europe from [this page](https://www.topuniversities.com/university-rankings/world-university-rankings/2020), and wrote a short Perl script to perform some cleanup, because Perl is dead good at that.

In [77]:
universities = pd.read_csv("/home/jeremy/Documents/Kaggle Courses/Geospatial Analysis/top_universities.csv")
universities.head()
universities_copy = universities.copy(deep=True)

In [78]:
def my_geocoder(row):
    try:
        point = geocode(row, provider='nominatim').geometry.iloc[0]
        return pd.Series({'Latitude': point.y, 'Longitude': point.x, 'geometry': point})
    except:
        return None

universities[['Latitude', 'Longitude', 'geometry']] = universities.apply(lambda x: my_geocoder(x['Name']), axis=1)

print("{}% of addresses were geocoded!".format(
    (1 - sum(np.isnan(universities["Latitude"])) / len(universities)) * 100))

# Drop universities that were not successfully geocoded
universities = universities.loc[~np.isnan(universities["Latitude"])]
universities = gpd.GeoDataFrame(universities, geometry=universities.geometry)
universities.crs = {'init': 'epsg:4326'}
universities.head()

92.0% of addresses were geocoded!


,Rank,Name,Country,Latitude,Longitude,geometry
0,4,University of Oxford,United Kingdom,51.753454,-1.254010,POINT (-1.25401 51.75345)
1,6,ETH Zurich - Swiss Federal Institute of Techno...,Switzerland,47.376453,8.547709,POINT (8.54771 47.37645)
2,7,University of Cambridge,United Kingdom,52.201828,0.109664,POINT (0.10966 52.20183)
3,8,UCL,United Kingdom,51.524443,-0.133517,POINT (-0.13352 51.52444)
4,9,Imperial College London,United Kingdom,51.498871,-0.175608,POINT (-0.17561 51.49887)


In [79]:
# Function for displaying the map
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

In [80]:
# Create a map
m = folium.Map(location=[54, 15], tiles='openstreetmap', zoom_start=2)

# Add points to the map
for idx, row in universities.iterrows():
    Marker([row['Latitude'], row['Longitude']], popup=row['Name']).add_to(m)

# Display the map
embed_map(m, 'm.html')

Found national boundary data on [Natural Earth](https://www.naturalearthdata.com/downloads/50m-cultural-vectors/):

In [111]:
country_boundaries = gpd.read_file('/home/jeremy/Documents/Kaggle Courses/Geospatial Analysis/ne_50m_admin_0_countries/ne_50m_admin_0_countries.shp')

In [112]:
europe_boundaries = country_boundaries.loc[country_boundaries['CONTINENT'] == 'Europe']

In [113]:
europe = europe_boundaries#[europe_boundaries.geometry.type.isin('Polygon','Multipolygon')]

In [153]:
# Make the thing we're plotting a measure of density and not simply population.
# I read this is a good idea.
plot_dict = europe.copy(deep=True)
plot_dict = plot_dict.set_index(['NAME'])
plot_dict['population_density'] = plot_dict['POP_EST'] / plot_dict.area
plot_dict = plot_dict['population_density']
plot_dict.head()

NAME
Vatican           1.302660e+07
Jersey            6.319410e+06
Guernsey          1.111980e+07
Isle of Man       1.151820e+06
United Kingdom    1.957080e+06
Name: population_density, dtype: float64

In [151]:
m_6 = folium.Map(location=[49.1118, 9.5509], tiles='cartodbpositron', zoom_start=4)

# Add a choropleth map to the base map
Choropleth(geo_data=europe.__geo_interface__, 
           data=plot_dict,
           key_on="feature.id", #<--I think this is how it's supposed to work? I'm not seeing colors.
           fill_color='YlGnBu', 
           legend_name='Population density of European countries'
          ).add_to(m_6)

# Display the map
embed_map(m_6, 'm_6.html')

In [154]:
# Use spatial join to match universities to countries in Europe
european_universities = gpd.sjoin(universities, europe)

AttributeError: 'NoneType' object has no attribute 'intersection'

In [155]:
# Investigate the result
print("We located {} universities.".format(len(universities)))
print("Only {} of the universities were located in Europe (in {} different countries).".format(
    len(european_universities), len(european_universities.name.unique())))

european_universities.head()

We located 92 universities.


NameError: name 'european_universities' is not defined